[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/IvanNece/Detection-of-Anomalies-with-Localization/blob/PatchCore/notebooks/04_patchcore_clean.ipynb)

# PatchCore Clean Domain Training


In [ ]:
# ============================================================
# SETUP - Mount Google Drive & Clone Repository
# ============================================================

from google.colab import drive
from pathlib import Path
import os
import sys

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')
print("Done!\n")

# Clone repository on PatchCore branch
print("Cloning repository (branch: PatchCore)...")
repo_dir = '/content/Detection-of-Anomalies-with-Localization'

# Remove if exists
if os.path.exists(repo_dir):
    print("Removing existing repository...")
    !rm -rf {repo_dir}

# Clone with specific branch
!git clone -b PatchCore https://github.com/IvanNece/Detection-of-Anomalies-with-Localization.git {repo_dir}
print("Done!\n")

# Setup paths
PROJECT_ROOT = Path(repo_dir)

# Dataset location (direct from Drive, no duplication)
DATASET_PATH = Path('/content/drive/MyDrive/mvtec_ad')

# Output directories
MODELS_DIR = PROJECT_ROOT / 'outputs' / 'models'
RESULTS_DIR = PROJECT_ROOT / 'outputs' / 'results'
VIZ_DIR = PROJECT_ROOT / 'outputs' / 'visualizations'

MODELS_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
VIZ_DIR.mkdir(parents=True, exist_ok=True)

# Verify dataset exists
if not DATASET_PATH.exists():
    raise FileNotFoundError(
        f"Dataset not found at {DATASET_PATH}\n"
        f"Please ensure mvtec_ad folder is in your Google Drive root."
    )

# Add project root to Python path
sys.path.insert(0, str(PROJECT_ROOT))

print("\n" + "="*70)
print("SETUP COMPLETE")
print("="*70)
print(f"Project:  {PROJECT_ROOT}")
print(f"Dataset:  {DATASET_PATH}")
print(f"Branch:   PatchCore")
print(f"Models:   {MODELS_DIR}")
print(f"Results:  {RESULTS_DIR}")
print(f"Viz:      {VIZ_DIR}")
print("="*70)

In [ ]:
# ============================================================
# INSTALL FAISS - MUST BE DONE BEFORE IMPORTS!
# ============================================================
# FAISS speeds up coreset sampling by 10-100x

!pip install faiss-cpu --quiet

# Verify installation
try:
    import faiss
    print("✓ FAISS installed successfully!")
    print(f" FAISS version: {faiss.__version__}")
    print("\n  IMPORTANT: Now you can proceed with imports")
except ImportError:
    print(" FAISS installation failed, will use numpy fallback (VERY SLOW)")
    print("   Try running: !pip install faiss-cpu")

## Import Libraries

In [ ]:
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import time

from src.utils.reproducibility import set_seed
from src.utils.config import load_config
from src.utils.paths import ProjectPaths
from src.data.dataset import MVTecDataset
from src.data.transforms import get_clean_transforms
from src.data.splitter import load_splits
from src.models.patchcore import PatchCore

# Set seed for reproducibility
set_seed(42)

# Load configuration
config = load_config(PROJECT_ROOT / 'configs' / 'experiment_config.yaml')
paths = ProjectPaths(PROJECT_ROOT)

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}")

In [ ]:
# PATCH: Fix dataset __getitem__ to handle errors properly
# This is needed because the GitHub code doesn't have error handling
from PIL import Image

def patched_getitem(self, idx):
    """Patched version with error handling."""
    image_path = self.images[idx]
    try:
        image = Image.open(image_path).convert('RGB')
    except Exception as e:
        raise RuntimeError(f"Failed to load image {image_path}: {e}")
    
    mask_path = self.masks[idx]
    mask = None
    
    if mask_path is not None:
        try:
            mask = Image.open(mask_path).convert('L')
        except Exception as e:
            raise RuntimeError(f"Failed to load mask {mask_path}: {e}")
    
    if self.transform is not None:
        try:
            image, mask = self.transform(image, mask)
        except Exception as e:
            raise RuntimeError(f"Transform failed for {image_path}: {e}")
    
    if image is None:
        raise RuntimeError(f"Transform returned None for image {image_path}")
    
    label = self.labels[idx]
    return image, mask, label, image_path

# Apply patch
MVTecDataset.__getitem__ = patched_getitem
print("✓ Dataset patched with error handling")

## Load Clean Splits

In [ ]:
# Load clean splits
splits = load_splits(paths.get_split_path('clean'))

print("Dataset splits loaded:")
print("=" * 70)
for class_name in config.dataset.classes:
    print(f"\n{class_name.upper()}:")
    for split_name in ['train', 'val', 'test']:
        split_data = splits[class_name][split_name]
        n_normal = sum(1 for l in split_data['labels'] if l == 0)
        n_anomalous = sum(1 for l in split_data['labels'] if l == 1)
        print(f"  {split_name:5s}: {len(split_data['labels']):4d} images "
              f"({n_normal:3d} normal, {n_anomalous:3d} anomalous)")

## Train PatchCore Models

In [ ]:
# Transform
transform = get_clean_transforms(image_size=config.dataset.image_size)

# Custom collate function per debug
def debug_collate_fn(batch):
    """Custom collate con debug per trovare None."""
    for i, item in enumerate(batch):
        if item is None:
            raise RuntimeError(f"Batch item {i} is None!")
        if not isinstance(item, tuple) or len(item) != 4:
            raise RuntimeError(f"Batch item {i} has wrong format: {type(item)}")
        img, mask, label, path = item
        if img is None:
            raise RuntimeError(f"Image is None for path: {path}")
    
    # Default collate
    import torch
    images = torch.stack([item[0] for item in batch])
    masks = [item[1] for item in batch]
    labels = torch.tensor([item[2] for item in batch])
    paths = [item[3] for item in batch]
    return images, masks, labels, paths

# Hyperparameters
CORESET_RATIO = config.patchcore.coreset_sampling_ratio
BATCH_SIZE = 8
NUM_WORKERS = 0  # Set to 0 per Colab

print("\nTraining Configuration:")
print("=" * 70)
print(f"Coreset ratio: {CORESET_RATIO*100:.1f}%")
print(f"Batch size: {BATCH_SIZE}")
print(f"Num workers: {NUM_WORKERS}")
print(f"Image size: {config.dataset.image_size}")
print(f"Backbone layers: {config.patchcore.layers}")
print(f"Patch size: {config.patchcore.patch_size}")
print(f"N neighbors: {config.patchcore.n_neighbors}")

In [ ]:
# Train models for each class
trained_models = {}
training_stats = {}

for class_name in config.dataset.classes:
    print("\n" + "=" * 70)
    print(f"Training PatchCore for: {class_name.upper()}")
    print("=" * 70)
    
    # Create train dataset (only normal images)
    train_split = splits[class_name]['train']
    train_dataset = MVTecDataset.from_split(
        train_split,
        transform=transform,
        phase='train'
    )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=False,  # Disabilitato per evitare warning
        collate_fn=debug_collate_fn  # Custom collate per debug
    )
    
    print(f"\nTrain dataset: {len(train_dataset)} images")
    
    # Initialize PatchCore
    model = PatchCore(
        backbone_layers=config.patchcore.layers,
        patch_size=config.patchcore.patch_size,
        coreset_ratio=CORESET_RATIO,
        n_neighbors=config.patchcore.n_neighbors,
        device=device
    )
    
    # Fit model
    start_time = time.time()
    model.fit(train_loader, apply_coreset=True)
    training_time = time.time() - start_time
    
    # Save model
    model.save(paths.MODELS, class_name, domain='clean')
    
    # Store statistics
    training_stats[class_name] = {
        'n_train_images': len(train_dataset),
        'memory_bank_size': len(model.memory_bank),
        'training_time': training_time,
        'spatial_dims': model.spatial_dims
    }
    
    trained_models[class_name] = model
    
    print(f"\nCompleted {class_name.upper()}:")
    print(f"  Memory bank size: {len(model.memory_bank)}")
    print(f"  Training time: {training_time:.2f}s")
    print(f"  Spatial dims: {model.spatial_dims}")

print("\n" + "=" * 70)
print("All PatchCore models trained successfully!")
print("=" * 70)

## Training Statistics Summary

In [ ]:
import pandas as pd

# Create summary table
stats_df = pd.DataFrame(training_stats).T
stats_df['training_time'] = stats_df['training_time'].apply(lambda x: f"{x:.2f}s")

print("\nTraining Statistics Summary:")
print(stats_df.to_string())

# Save statistics
stats_output_path = paths.RESULTS / 'patchcore_clean_training_stats.csv'
stats_df.to_csv(stats_output_path)
print(f"\nStatistics saved to: {stats_output_path}")

## Quick Validation Test

Test predictions on a few validation images to verify the model works correctly.

In [ ]:
def denormalize_image(img_tensor, mean, std):
    """Denormalize image for visualization."""
    img = img_tensor.permute(1, 2, 0).numpy()
    img = img * np.array(std) + np.array(mean)
    img = np.clip(img, 0, 1)
    return img

# Test on validation set
class_name = 'hazelnut'  # Change to test different classes
model = trained_models[class_name]

# Load validation data
val_split = splits[class_name]['val']
val_dataset = MVTecDataset.from_split(
    val_split,
    transform=transform,
    phase='val'
)

# Select images: 4 normal + 4 anomalous
normal_idx = [i for i, l in enumerate(val_dataset.labels) if l == 0][:4]
anomalous_idx = [i for i, l in enumerate(val_dataset.labels) if l == 1][:4]
test_indices = normal_idx + anomalous_idx

# Get predictions
test_images = []
test_labels = []
for idx in test_indices:
    img, mask, label, _ = val_dataset[idx]
    test_images.append(img)
    test_labels.append(label)

test_batch = torch.stack(test_images)
scores, heatmaps = model.predict(test_batch, return_heatmaps=True)

print(f"\nPrediction scores for {class_name}:")
print(f"Normal images (0-3): {scores[:4]}")
print(f"Anomalous images (4-7): {scores[4:]}")
print(f"\nScore statistics:")
print(f"  Normal - mean: {scores[:4].mean():.3f}, std: {scores[:4].std():.3f}")
print(f"  Anomalous - mean: {scores[4:].mean():.3f}, std: {scores[4:].std():.3f}")

## Visualize Predictions

In [ ]:
# Visualization
fig, axes = plt.subplots(2, 8, figsize=(20, 5))

mean = config.dataset.normalize.mean
std = config.dataset.normalize.std

# Calculate global vmax for consistent heatmap scaling
vmax_global = np.max([h.max() for h in heatmaps])
print(f"Global heatmap vmax: {vmax_global:.3f}")

for i, (img, score, heatmap, label) in enumerate(
    zip(test_images, scores, heatmaps, test_labels)
):
    # Denormalize image
    img_np = denormalize_image(img, mean, std)
    
    # Original image
    axes[0, i].imshow(img_np)
    axes[0, i].set_title(
        f"{'Normal' if label == 0 else 'Anomalous'}\nScore: {score:.3f}",
        fontsize=10
    )
    axes[0, i].axis('off')
    
    # Heatmap overlay with global normalization
    axes[1, i].imshow(img_np)
    axes[1, i].imshow(heatmap, alpha=0.5, cmap='jet', vmin=0, vmax=vmax_global)
    axes[1, i].axis('off')

axes[0, 0].set_ylabel('Original', fontsize=12)
axes[1, 0].set_ylabel('Heatmap', fontsize=12)

plt.show()

plt.suptitle(f'PatchCore Predictions - {class_name.upper()}', fontsize=16, y=1.02)plt.tight_layout()

## Save Visualizations

In [ ]:
# Save visualization for each class
for class_name in config.dataset.classes:
    model = trained_models[class_name]
    val_split = splits[class_name]['val']
    val_dataset = MVTecDataset.from_split(
        val_split, transform=transform, phase='val'
    )
    
    # Select images
    normal_idx = [i for i, l in enumerate(val_dataset.labels) if l == 0][:4]
    anomalous_idx = [i for i, l in enumerate(val_dataset.labels) if l == 1][:4]
    test_indices = normal_idx + anomalous_idx
    
    test_images = []
    test_labels = []
    for idx in test_indices:
        img, _, label, _ = val_dataset[idx]
        test_images.append(img)
        test_labels.append(label)
    
    test_batch = torch.stack(test_images)
    scores, heatmaps = model.predict(test_batch, return_heatmaps=True)
    
    # Calculate global vmax for consistent heatmap scaling
    vmax_global = np.max([h.max() for h in heatmaps])
    
    # Create figure
    fig, axes = plt.subplots(2, 8, figsize=(20, 5))
    
    for i, (img, score, heatmap, label) in enumerate(
        zip(test_images, scores, heatmaps, test_labels)
    ):
        img_np = denormalize_image(img, mean, std)
        
        axes[0, i].imshow(img_np)
        axes[0, i].set_title(
            f"{'Normal' if label == 0 else 'Anomalous'}\nScore: {score:.3f}",
            fontsize=10
        )
        axes[0, i].axis('off')
        
        axes[1, i].imshow(img_np)
        axes[1, i].imshow(heatmap, alpha=0.5, cmap='jet', vmin=0, vmax=vmax_global)
        axes[1, i].axis('off')
    
    axes[0, 0].set_ylabel('Original', fontsize=12)
    axes[1, 0].set_ylabel('Heatmap', fontsize=12)
    
    plt.suptitle(f'PatchCore Predictions - {class_name.upper()}', fontsize=16, y=1.02)
    plt.tight_layout()
    
    # Save
    save_path = paths.VISUALIZATIONS / f'patchcore_clean_{class_name}_validation.png'
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.close()
    

    print(f"Saved visualization: {save_path}")print("\nAll visualizations saved!")


## Summary

Phase 3 completed successfully:
- Trained PatchCore models for all classes
- Memory banks saved in `outputs/models/`
- Validation predictions show clear separation between normal and anomalous
- Ready for Phase 5: Threshold calibration and full evaluation

## Save Complete Training Metadata

Save comprehensive metadata for reproducibility and downstream evaluation.

In [ ]:
import json
from datetime import datetime

# Collect validation scores for threshold calibration
validation_predictions = {}

for class_name in config.dataset.classes:
    model = trained_models[class_name]
    
    # Load validation set
    val_split = splits[class_name]['val']
    val_dataset = MVTecDataset.from_split(
        val_split, transform=transform, phase='val'
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=False,
        collate_fn=debug_collate_fn  # Use custom collate to handle None masks
    )
    
    # Get predictions for all validation images
    all_scores = []
    all_labels = []
    all_paths = []
    
    for images, masks, labels, image_paths in val_loader:
        scores, _ = model.predict(images, return_heatmaps=False)
        
        all_scores.extend(scores.tolist())
        all_labels.extend(labels.tolist())
        all_paths.extend(image_paths)
    
    validation_predictions[class_name] = {
        'scores': all_scores,
        'labels': all_labels,
        'image_paths': all_paths,
        'n_normal': sum(1 for l in all_labels if l == 0),
        'n_anomalous': sum(1 for l in all_labels if l == 1),
        'score_stats': {
            'normal_mean': float(np.mean([s for s, l in zip(all_scores, all_labels) if l == 0])),
            'normal_std': float(np.std([s for s, l in zip(all_scores, all_labels) if l == 0])),
            'anomalous_mean': float(np.mean([s for s, l in zip(all_scores, all_labels) if l == 1])),
            'anomalous_std': float(np.std([s for s, l in zip(all_scores, all_labels) if l == 1]))
        }
    }
    
    print(f"{class_name}: Normal scores: {validation_predictions[class_name]['score_stats']['normal_mean']:.3f} +/- "
          f"{validation_predictions[class_name]['score_stats']['normal_std']:.3f}, "
          f"Anomalous: {validation_predictions[class_name]['score_stats']['anomalous_mean']:.3f} +/- "
          f"{validation_predictions[class_name]['score_stats']['anomalous_std']:.3f}")

In [ ]:
# Ensure paths object is not corrupted (safety check)
if not hasattr(paths, 'MODELS'):
    paths = ProjectPaths(PROJECT_ROOT)

# Create comprehensive metadata
metadata = {
    'notebook': '04_patchcore_clean.ipynb',
    'timestamp': datetime.now().isoformat(),
    'phase': 'Phase 3 - PatchCore Clean Domain',
    'seed': 42,
    'device': device,
    
    'configuration': {
        'backbone': config.patchcore.backbone,
        'layers': config.patchcore.layers,
        'patch_size': config.patchcore.patch_size,
        'coreset_ratio': CORESET_RATIO,
        'n_neighbors': config.patchcore.n_neighbors,
        'batch_size': BATCH_SIZE,
        'image_size': config.dataset.image_size,
        'normalize_mean': config.dataset.normalize.mean,
        'normalize_std': config.dataset.normalize.std
    },
    
    'training_statistics': {},
    'validation_predictions': validation_predictions,
    
    'models_saved': {},
    'splits_used': 'clean_splits.json'
}

# Add training statistics per class
for class_name in config.dataset.classes:
    stats = training_stats[class_name]
    metadata['training_statistics'][class_name] = {
        'n_train_images': stats['n_train_images'],
        'memory_bank_size': stats['memory_bank_size'],
        'training_time_seconds': stats['training_time'],
        'spatial_dims': stats['spatial_dims'],
        'compression_ratio': stats['memory_bank_size'] / (stats['n_train_images'] * stats['spatial_dims'][0] * stats['spatial_dims'][1]) if stats['n_train_images'] > 0 else 0
    }
    
    metadata['models_saved'][class_name] = {
        'memory_bank': f"patchcore_{class_name}_clean.npy",
        'config': f"patchcore_{class_name}_clean_config.pth",
        'location': str(paths.MODELS)
    }

# Save metadata
metadata_path = paths.RESULTS / 'patchcore_clean_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"\nMetadata saved to: {metadata_path}")
print(f"\nSummary:")
print(f"  Classes trained: {len(config.dataset.classes)}")
print(f"  Total memory bank samples: {sum(stats['memory_bank_size'] for stats in training_stats.values())}")
print(f"  Average training time: {np.mean([stats['training_time'] for stats in training_stats.values()]):.2f}s")
print(f"  Models saved in: {paths.MODELS}")
print(f"  Results saved in: {paths.RESULTS}")
print(f"  Visualizations saved in: {paths.VISUALIZATIONS}")

In [ ]:
# Display final summary table
print("\n" + "="*80)
print("PHASE 3 COMPLETED - PATCHCORE CLEAN DOMAIN TRAINING")
print("="*80)

summary_data = []
for class_name in config.dataset.classes:
    stats = training_stats[class_name]
    val_stats = validation_predictions[class_name]['score_stats']
    
    summary_data.append({
        'Class': class_name.upper(),
        'Train Images': stats['n_train_images'],
        'Memory Bank': stats['memory_bank_size'],
        'Train Time (s)': f"{stats['training_time']:.2f}",
        'Normal Score': f"{val_stats['normal_mean']:.3f}±{val_stats['normal_std']:.3f}",
        'Anomalous Score': f"{val_stats['anomalous_mean']:.3f}±{val_stats['anomalous_std']:.3f}",
        'Separation': f"{val_stats['anomalous_mean'] - val_stats['normal_mean']:.3f}"
    })

summary_df = pd.DataFrame(summary_data)
print("\n", summary_df.to_string(index=False))

print("\n" + "="*80)
print("Next Steps:")
print("  1. Phase 4: Train PaDiM baseline (notebook 05)")
print("  2. Phase 5: Threshold calibration and evaluation (notebook 06)")
print("  3. Use validation scores for optimal threshold selection")
print("="*80)

## Files Generated

This notebook generated the following files:

**Models** (`outputs/models/`):
- `patchcore_hazelnut_clean.npy` - Memory bank
- `patchcore_hazelnut_clean_config.pth` - Model config
- `patchcore_carpet_clean.npy` - Memory bank
- `patchcore_carpet_clean_config.pth` - Model config
- `patchcore_zipper_clean.npy` - Memory bank
- `patchcore_zipper_clean_config.pth` - Model config

**Results** (`outputs/results/`):
- `patchcore_clean_training_stats.csv` - Training statistics
- `patchcore_clean_metadata.json` - Complete metadata with validation scores

**Visualizations** (`outputs/visualizations/`):
- `patchcore_clean_hazelnut_validation.png` - Validation predictions
- `patchcore_clean_carpet_validation.png` - Validation predictions
- `patchcore_clean_zipper_validation.png` - Validation predictions

All files are ready for Phase 5 (evaluation) and Phase 6-7 (domain shift experiments).

## Save Outputs to Google Drive

In [ ]:
import shutil
from datetime import datetime

# Create output directory on Drive
DRIVE_OUTPUT = Path('/content/drive/MyDrive/anomaly_detection_project/04_patchcore_clean_outputs')
DRIVE_OUTPUT.mkdir(parents=True, exist_ok=True)

print("Saving outputs to Google Drive...")
print("="*70)

# 1. Copy model files (memory banks + configs)
models_saved = []
for class_name in config.dataset.classes:
    memory_bank = paths.MODELS / f"patchcore_{class_name}_clean.npy"
    model_config = paths.MODELS / f"patchcore_{class_name}_clean_config.pth"
    
    if memory_bank.exists():
        shutil.copy2(memory_bank, DRIVE_OUTPUT / memory_bank.name)
        models_saved.append(memory_bank.name)
        print(f"✓ Saved: {memory_bank.name}")
    
    if model_config.exists():
        shutil.copy2(model_config, DRIVE_OUTPUT / model_config.name)
        models_saved.append(model_config.name)
        print(f"✓ Saved: {model_config.name}")

# 2. Copy results files
results_files = [
    (paths.RESULTS / 'patchcore_clean_training_stats.csv', 'patchcore_clean_training_stats.csv'),
    (paths.RESULTS / 'patchcore_clean_metadata.json', 'patchcore_clean_metadata.json')
]

for src, dst_name in results_files:
    if src.exists():
        shutil.copy2(src, DRIVE_OUTPUT / dst_name)
        print(f"✓ Saved: {dst_name}")

# 3. Copy visualization files
viz_files = []
for class_name in config.dataset.classes:
    viz_file = paths.VISUALIZATIONS / f"patchcore_clean_{class_name}_validation.png"
    if viz_file.exists():
        shutil.copy2(viz_file, DRIVE_OUTPUT / viz_file.name)
        viz_files.append(viz_file.name)
        print(f"✓ Saved: {viz_file.name}")

print("\n" + "="*70)
print("OUTPUTS SAVED TO GOOGLE DRIVE!")
print("="*70)
print(f"Location: {DRIVE_OUTPUT}")
print(f"Total files: {len(list(DRIVE_OUTPUT.iterdir()))}")
print(f"Total size: {sum(f.stat().st_size for f in DRIVE_OUTPUT.iterdir() if f.is_file()) / (1024*1024):.2f} MB")
print("="*70)

## Create ZIP Archive

In [ ]:
# Create zip archive with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
zip_name = f'04_patchcore_clean_outputs_{timestamp}'
zip_path = Path('/content/drive/MyDrive/anomaly_detection_project') / zip_name

print(f"Creating zip archive...")
print(f"Source: {DRIVE_OUTPUT}")
print(f"Archive: {zip_path}.zip")
print("\nThis may take a minute...")

# Create zip
shutil.make_archive(str(zip_path), 'zip', DRIVE_OUTPUT)

zip_size = (Path(str(zip_path) + '.zip')).stat().st_size / (1024*1024)

print("\n" + "="*70)
print("ZIP ARCHIVE CREATED!")
print("="*70)
print(f"Location: {zip_path}.zip")
print(f"Size: {zip_size:.2f} MB")
print(f"\nContents:")
print(f"  - {len(models_saved)} model files (.npy + .pth)")
print(f"  - {len(results_files)} result files (.csv, .json)")
print(f"  - {len(viz_files)} visualizations (.png)")
print("="*70)

## Download ZIP Locally (Optional)

In [ ]:
# Optional: Download zip to local machine
from google.colab import files

# Create a local copy of the zip
local_zip = f'/content/{zip_name}.zip'
shutil.copy2(str(zip_path) + '.zip', local_zip)

print("Downloading ZIP to your local machine...")
print(f"File: {local_zip}")
print(f"Size: {zip_size:.2f} MB")
print("\nDownload will start automatically...")

files.download(local_zip)

print("\n" + "="*70)
print("DOWNLOAD COMPLETE!")
print("="*70)
print("\nNext steps:")
print("  1. Extract ZIP file locally")
print("  2. Copy files to your local project's outputs/ folder")
print("  3. Review results and visualizations")
print("  4. Proceed with Phase 4: PaDiM baseline (notebook 05)")
print("="*70)